In [0]:
#!pip install gensim
import gensim

In [0]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', 
                    level=logging.INFO)

In [3]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/FFR/FFR-MLData.csv')

print('Number of examples in Dataset: ', df.shape)
df.head()

Number of examples in Dataset:  (8512, 3)


,Fault Description,Module,Sub-module
0,BATTERY BACKUP LOW,Power,Battery Drains
1,POWER BUTTON NOT WORKING,Hardware,Button
2,MIC PROBLEM,Hardware,Mic
3,S/W PROBLEM,Others,NaN
4,"CALL FORWARDING, CONFERENCE CALL NOT WORKING, ...",Protocol,call-conference call


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8512 entries, 0 to 8511
Data columns (total 3 columns):
Fault Description    8509 non-null object
Module               8495 non-null object
Sub-module           7850 non-null object
dtypes: object(3)
memory usage: 199.6+ KB


In [0]:
df.dropna(inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7850 entries, 0 to 8511
Data columns (total 3 columns):
Fault Description    7850 non-null object
Module               7850 non-null object
Sub-module           7850 non-null object
dtypes: object(3)
memory usage: 245.3+ KB


In [9]:
df['Module'].value_counts()

Power           1741
Performance     1623
Hardware        1228
System           994
MM               786
Protocol         603
FRP              364
Sensor           197
Third-party      123
Connectivity      90
System apps       70
Settings          25
system             4
System             1
hardware           1
Name: Module, dtype: int64

In [11]:
df.drop(df[df.Module == 'hardware'].index, inplace=True)
df['Module'].value_counts()

Power           1741
Performance     1623
Hardware        1228
System           994
MM               786
Protocol         603
FRP              364
Sensor           197
Third-party      123
Connectivity      90
System apps       70
Settings          25
system             4
System             1
Name: Module, dtype: int64

In [67]:
df.drop(df[df.Module == 'system'].index, inplace=True)
df.drop(df[df.Module == 'System'].index, inplace=True)
df.drop(df[df.Module == 'System '].index, inplace=True)
df.drop(df[df.Module == 'Settings'].index, inplace=True)
df.drop(df[df.Module == 'System apps'].index, inplace=True)
df.drop(df[df.Module == 'Connectivity'].index, inplace=True)
df.drop(df[df.Module == 'Sensor'].index, inplace=True)
df.drop(df[df.Module == 'Third-party'].index, inplace=True)
df['Module'].value_counts()

Power          1741
Performance    1623
Hardware       1228
MM              786
Protocol        603
FRP             364
Name: Module, dtype: int64

In [68]:
df['ExtMoule'] = df['Module'] + '-' + df['Sub-module']
df.head()

,Fault Description,Module,Sub-module,ExtMoule,Text
0,BATTERY BACKUP LOW,Power,Battery Drains,Power-Battery Drains,battery backup low
1,POWER BUTTON NOT WORKING,Hardware,Button,Hardware-Button,power button not working
2,MIC PROBLEM,Hardware,Mic,Hardware-Mic,mic problem
4,"CALL FORWARDING, CONFERENCE CALL NOT WORKING, ...",Protocol,call-conference call,Protocol-call-conference call,call forwarding conference call not working ha...
5,"CALL FORWARDING, CONFERENCE CALL NOT WORKING, ...",Performance,Device hanging,Performance-Device hanging,call forwarding conference call not working ha...


In [0]:
# Observation: Notonly Module Sub-Module is too scattered. Stict to Module for first level of implementation.

In [0]:
import re, string

def clean_str(string):
  """
  String cleaning before vectorization
  """
  try:    
    string = re.sub(r'^https?:\/\/<>.*[\r\n]*', '', string, flags=re.MULTILINE)
    string = re.sub(r"[^A-Za-z]", " ", string)         
    words = string.strip().lower().split()    
    words = [w for w in words if len(w)>=1]
    return " ".join(words)	
  except:
    return ""

In [71]:
df['Text'] = df['Fault Description'].apply(clean_str)
df.head()

,Fault Description,Module,Sub-module,ExtMoule,Text
0,BATTERY BACKUP LOW,Power,Battery Drains,Power-Battery Drains,battery backup low
1,POWER BUTTON NOT WORKING,Hardware,Button,Hardware-Button,power button not working
2,MIC PROBLEM,Hardware,Mic,Hardware-Mic,mic problem
4,"CALL FORWARDING, CONFERENCE CALL NOT WORKING, ...",Protocol,call-conference call,Protocol-call-conference call,call forwarding conference call not working ha...
5,"CALL FORWARDING, CONFERENCE CALL NOT WORKING, ...",Performance,Device hanging,Performance-Device hanging,call forwarding conference call not working ha...


In [72]:
#List to hold all words in each review
documents = []

#Iterate over each review
for doc in df['Text']:
    documents.append(doc.split(' '))

print(len(documents))
print(documents[0])

6345
['battery', 'backup', 'low']


In [73]:
#Build the model
model = gensim.models.Word2Vec(documents, #Word list
                               min_count=5, #Ignore all words with total frequency lower than this                           
                               workers=4, #Number of CPUs
                               size=50,  #Embedding size
                               window=3, #Maximum Distance between current and predicted word
                               iter=100   #Number of iterations over the text corpus
                              )  

2018-12-12 03:14:01,260 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-12-12 03:14:01,262 : INFO : collecting all words and their counts
2018-12-12 03:14:01,263 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-12 03:14:01,273 : INFO : collected 1689 word types from a corpus of 32595 raw words and 6345 sentences
2018-12-12 03:14:01,274 : INFO : Loading a fresh vocabulary
2018-12-12 03:14:01,277 : INFO : effective_min_count=5 retains 469 unique words (27% of original 1689, drops 1220)
2018-12-12 03:14:01,278 : INFO : effective_min_count=5 leaves 30625 word corpus (93% of original 32595, drops 1970)
2018-12-12 03:14:01,281 : INFO : deleting the raw counts dictionary of 1689 items
2018-12-12 03:14:01,282 : INFO : sample=0.001 downsamples 74 most-common words
2018-12-12 03:14:01,283 : INFO : downsampling leaves estimated 16393 word corpus (53.5% of prior 30625)
2018-12-12 03:14:01,285 : INFO : estimated required memo

In [74]:
#Model size
print(model.wv.syn0.shape)

(469, 50)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [0]:
words = []
for word in model.wv.vocab:
    words.append(word)

In [76]:
import numpy as np
 
def sent_vectorizer(sent, model):
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model.wv[w]
            else:
                sent_vec = np.add(sent_vec, model[w])
            numw+=1
        except:
            pass
    
    return np.asarray(sent_vec) / numw
 
 
V=[]
for sentence in df['Text']:
    V.append(sent_vectorizer(sentence, model))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [77]:
print(len(V))

6345


In [78]:
print(V[0].shape)

(50,)


In [79]:
print(V[0])

[ 0.08242327 -1.395165   -0.90242064  0.01375185  0.9656248  -0.2508595
 -0.63607234  0.82382154 -0.15963206 -0.5739207   0.18542391  0.05185815
  0.41369852  0.7819859   0.79899824 -0.26532692  0.1748614  -1.403486
  0.1303905   0.33180895 -0.61118335  0.4729473   0.15106268 -0.3860006
  0.40313774  0.39843225 -0.1302468   0.44230583  0.6897683  -0.53028643
 -0.5804054  -0.5502048   1.1582547   0.04483563 -0.33598772  0.45470732
  0.6765277   0.00527252  0.00189841 -0.3733431   0.4839646  -0.04817668
 -0.7198341   0.9213735   0.29206017 -0.39428848 -0.6837517   0.8113009
 -0.36223823 -0.7752373 ]


In [0]:
df.loc[df['Module'].isin(['System ','system'])] = 'System'

In [81]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y = le.fit_transform(df['Module'])
Y_classes = le.classes_
print("Classes:", Y_classes)
print(len(Y))

Classes: ['FRP' 'Hardware' 'MM' 'Performance' 'Power' 'Protocol']
6345


In [119]:
fdf = pd.DataFrame(data=V)
fdf.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.082423,-1.395165,-0.902421,0.013752,0.965625,-0.250859,-0.636072,0.823822,-0.159632,-0.573921,...,0.483965,-0.048177,-0.719834,0.921373,0.292060,-0.394288,-0.683752,0.811301,-0.362238,-0.775237
1,-0.068175,-1.272794,-1.019871,-0.178811,0.532213,-0.041718,-0.845634,0.678139,0.201283,0.091480,...,0.626224,-0.273176,-0.761032,1.045175,-0.419423,-0.589640,-0.169023,0.303231,0.218113,-0.422987
2,-0.422800,-0.969194,-0.586218,0.077776,0.962181,0.570693,-1.417914,0.217061,-0.571257,0.523616,...,0.149448,0.237225,-0.846916,1.534863,0.045520,-0.868786,-0.325854,1.100540,-0.434048,0.295558
3,-0.272435,-0.813553,-0.871080,0.331175,0.784399,0.323660,-0.818591,0.402451,-0.112049,-0.313358,...,0.277199,-0.081505,-0.919997,1.008627,-0.155427,-0.493912,-0.140835,0.325116,-0.210447,-0.443485
4,-0.272435,-0.813553,-0.871080,0.331175,0.784399,0.323660,-0.818591,0.402451,-0.112049,-0.313358,...,0.277199,-0.081505,-0.919997,1.008627,-0.155427,-0.493912,-0.140835,0.325116,-0.210447,-0.443485


In [0]:
fdf['class'] = Y

In [121]:
fdf.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,class
0,0.082423,-1.395165,-0.902421,0.013752,0.965625,-0.250859,-0.636072,0.823822,-0.159632,-0.573921,...,-0.048177,-0.719834,0.921373,0.292060,-0.394288,-0.683752,0.811301,-0.362238,-0.775237,4
1,-0.068175,-1.272794,-1.019871,-0.178811,0.532213,-0.041718,-0.845634,0.678139,0.201283,0.091480,...,-0.273176,-0.761032,1.045175,-0.419423,-0.589640,-0.169023,0.303231,0.218113,-0.422987,1
2,-0.422800,-0.969194,-0.586218,0.077776,0.962181,0.570693,-1.417914,0.217061,-0.571257,0.523616,...,0.237225,-0.846916,1.534863,0.045520,-0.868786,-0.325854,1.100540,-0.434048,0.295558,1
3,-0.272435,-0.813553,-0.871080,0.331175,0.784399,0.323660,-0.818591,0.402451,-0.112049,-0.313358,...,-0.081505,-0.919997,1.008627,-0.155427,-0.493912,-0.140835,0.325116,-0.210447,-0.443485,5
4,-0.272435,-0.813553,-0.871080,0.331175,0.784399,0.323660,-0.818591,0.402451,-0.112049,-0.313358,...,-0.081505,-0.919997,1.008627,-0.155427,-0.493912,-0.140835,0.325116,-0.210447,-0.443485,3


In [0]:
def split_dataframe(df, Y, factor=0.8, seed=7):
    """ Split the given data frame into training and data sets. """
    # Add Y to scale_ignore
    
    np.random.seed(seed=seed)
    msk = np.random.rand(len(df)) < factor
    train = df[msk]
    test = df[~msk]

    X_train = train.drop(Y, axis=1).values
    Y_train = train[Y].values

    X_test = test.drop(Y, axis=1).values
    Y_test = test[Y].values

    print(X_train.shape)
    print(X_test.shape)
    print(Y_train.shape)
    print(Y_test.shape)

    return (X_train, Y_train, X_test, Y_test)

In [123]:
(X_train, Y_train, X_test, Y_test) = split_dataframe(fdf, 'class')

(5068, 50)
(1277, 50)
(5068,)
(1277,)


In [0]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)  

In [0]:
from sklearn.ensemble import RandomForestClassifier

rmodel = RandomForestClassifier(n_estimators=100, max_depth=2,
                                random_state=0)
rmodel.fit(X_train, Y_train)
y_pred = rmodel.predict(X_test)

In [126]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print ("Train Accuracy :: ", accuracy_score(Y_train, rmodel.predict(X_train)))
print ("Test Accuracy  :: ", accuracy_score(Y_test, y_pred))

Train Accuracy ::  0.5045382794001578
Test Accuracy  ::  0.4988253719655442


In [0]:
import tensorflow as tf
import keras as keras

In [0]:
trainY = keras.utils.to_categorical(Y_train, num_classes=len(Y_classes))
testY = keras.utils.to_categorical(Y_test, num_classes=len(Y_classes))

In [0]:
#Initialize Sequential model
model = keras.models.Sequential()

model.add(keras.layers.Reshape((50,),input_shape=(50,)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(200, activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(len(Y_classes), activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [130]:
model.fit(X_train, trainY, 
          validation_data=(X_test, testY), 
          epochs=500,
          batch_size=100)

Train on 5068 samples, validate on 1277 samples
Epoch 1/500
5068/5068 [==============================] - 1s 130us/step - loss: 1.7155 - acc: 0.3337 - val_loss: 1.4281 - val_acc: 0.4401
Epoch 2/500
5068/5068 [==============================] - 0s 56us/step - loss: 1.3865 - acc: 0.4702 - val_loss: 1.2718 - val_acc: 0.5419
Epoch 3/500
5068/5068 [==============================] - 0s 55us/step - loss: 1.2752 - acc: 0.5217 - val_loss: 1.1955 - val_acc: 0.5724
Epoch 4/500
5068/5068 [==============================] - 0s 55us/step - loss: 1.2222 - acc: 0.5468 - val_loss: 1.1485 - val_acc: 0.5779
Epoch 5/500
5068/5068 [==============================] - 0s 57us/step - loss: 1.1731 - acc: 0.5566 - val_loss: 1.1149 - val_acc: 0.5771
Epoch 6/500
5068/5068 [==============================] - 0s 57us/step - loss: 1.1542 - acc: 0.5641 - val_loss: 1.0883 - val_acc: 0.5826
Epoch 7/500
5068/5068 [==============================] - 0s 57us/step - loss: 1.1244 - acc: 0.5724 - val_loss: 1.0676 - val_acc: 0.5951

In [131]:
y_pred = model.predict(X_test)
y_pred.shape

(1277, 6)

In [0]:
y_pred_1 = keras.backend.argmax(y_pred)
y_pred_2 = keras.backend.eval(y_pred_1)

In [133]:
keras.backend.eval(tf.confusion_matrix(
    Y_test,
    y_pred_2,
    num_classes=len(Y_classes),
    dtype=tf.int32,
    name=None,
    weights=None
))

array([[ 67,   4,   0,   2,   2,   1],
       [  2, 170,  17,  12,  27,  17],
       [  0,  16, 101,  14,  24,   6],
       [  0,   6,   6, 220,  62,   5],
       [  0,  13,  14,  50, 278,   5],
       [  2,  26,  12,   9,  16,  71]], dtype=int32)

In [134]:
Y_classes

array(['FRP', 'Hardware', 'MM', 'Performance', 'Power', 'Protocol'],
      dtype=object)

In [135]:
np.bincount(Y_test)

array([ 76, 245, 161, 299, 360, 136])

In [137]:
print("FRP :", 67/76)
print("Hardware :", 170/245)
print("MM :", 101/161)
print("Performance :", 220/299)
print("Power :", 278/360)
print("Protocol :", 71/136)

FRP : 0.881578947368421
Hardware : 0.6938775510204082
MM : 0.6273291925465838
Performance : 0.7357859531772575
Power : 0.7722222222222223
Protocol : 0.5220588235294118
